In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional, Conv1D, MaxPooling1D, Flatten, Concatenate
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow.keras as keras

In [ ]:
X_train = pd.read_csv("./data/X_train_lstm.csv")
X_test = pd.read_csv("./data/X_test_lstm.csv")

Y_train = pd.read_csv("./data/Y_train_lstm.csv")
Y_test = pd.read_csv("./data/Y_test_lstm.csv")

### Set random seeds

In [3]:
np.random.seed(42)
tf.random.set_seed(42)
keras.utils.set_random_seed(42)

### Train LSTM

In [ ]:
# Defining callbacks
checkpoint = ModelCheckpoint("./models/lstm_model.keras", monitor='val_loss', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

# Define LSTM model
# Up to 2 layers of LSTM and number of hidden units were hand tuned to determine this as the optimum model
lstm_model = Sequential([
    Input(shape=(X_train.shape[1], 1)),
    LSTM(units=64, activation='relu', recurrent_dropout=0.2),
    Dense(5)
])

# Use MSE for loss because we want to emphasize the "wrongest" guesses the most. MAE is an interpretable metric
lstm_model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])

# Train model w/ early stopping
# Batch size is the average number of flights per day
history = lstm_model.fit(X_train, Y_train, epochs=50, batch_size=265, validation_split=0.2, callbacks=[checkpoint, early_stopping])


loss, mae = lstm_model.evaluate(X_test, Y_test)
print("Test Mean Absolute Error:", mae)

Y_pred = lstm_model.predict(X_test)

mae_columns = mean_absolute_error(Y_test, Y_pred, multioutput='raw_values')
print("Mean Absolute Error for each column:")
print(mae_columns)

Epoch 1/50
1110/1110 [==============================] - ETA: 0s - loss: 401.9368 - mae: 10.8933
Epoch 1: val_loss improved from inf to 378.16812, saving model to ../models\lstm_model.keras
1110/1110 [==============================] - 46s 40ms/step - loss: 401.9368 - mae: 10.8933 - val_loss: 378.1681 - val_mae: 10.1630
Epoch 2/50
1109/1110 [============================>.] - ETA: 0s - loss: 380.4790 - mae: 10.3173
Epoch 2: val_loss improved from 378.16812 to 374.56149, saving model to ../models\lstm_model.keras
1110/1110 [==============================] - 44s 40ms/step - loss: 380.4741 - mae: 10.3172 - val_loss: 374.5615 - val_mae: 9.9608
Epoch 3/50
1109/1110 [============================>.] - ETA: 0s - loss: 377.6657 - mae: 10.2380
Epoch 3: val_loss improved from 374.56149 to 372.89502, saving model to ../models\lstm_model.keras
1110/1110 [==============================] - 44s 40ms/step - loss: 377.6783 - mae: 10.2381 - val_loss: 372.8950 - val_mae: 10.2404
Epoch 4/50
1109/1110 [=======

### Train BiLSTM

In [ ]:
# Defining callbacks
checkpoint = ModelCheckpoint("./models/bilstm_model.keras", monitor='val_loss', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

# Define LSTM model
# Up to 2 layers of LSTM and number of hidden units were hand tuned to determine this as the optimum model
bilstm_model = Sequential([
    Input(shape=(X_train.shape[1], 1)),
    Bidirectional(
        LSTM(units=64, activation='relu', recurrent_dropout=0.2)
    ),
    Dense(5)
])

# Use MSE for loss because we want to emphasize the "wrongest" guesses the most. MAE is an interpretable metric
bilstm_model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])

# Train model w/ early stopping
# Batch size is the average number of flights per day
history = bilstm_model.fit(X_train, Y_train, epochs=50, batch_size=265, validation_split=0.2, callbacks=[checkpoint, early_stopping])


loss, mae = bilstm_model.evaluate(X_test, Y_test)
print("Test Mean Absolute Error:", mae)

Y_pred = bilstm_model.predict(X_test)

mae_columns = mean_absolute_error(Y_test, Y_pred, multioutput='raw_values')
print("Mean Absolute Error for each column:")
print(mae_columns)

Epoch 1/50
1110/1110 [==============================] - ETA: 0s - loss: 445.2512 - mae: 11.9186
Epoch 1: val_loss improved from inf to 378.54678, saving model to ../models\bilstm_model.keras
1110/1110 [==============================] - 86s 76ms/step - loss: 445.2512 - mae: 11.9186 - val_loss: 378.5468 - val_mae: 10.2095
Epoch 2/50
1110/1110 [==============================] - ETA: 0s - loss: 381.6242 - mae: 10.4476
Epoch 2: val_loss improved from 378.54678 to 374.90466, saving model to ../models\bilstm_model.keras
1110/1110 [==============================] - 84s 76ms/step - loss: 381.6242 - mae: 10.4476 - val_loss: 374.9047 - val_mae: 10.0181
Epoch 3/50
1110/1110 [==============================] - ETA: 0s - loss: 378.9670 - mae: 10.3367
Epoch 3: val_loss improved from 374.90466 to 374.22491, saving model to ../models\bilstm_model.keras
1110/1110 [==============================] - 84s 76ms/step - loss: 378.9670 - mae: 10.3367 - val_loss: 374.2249 - val_mae: 10.8192
Epoch 4/50
1110/1110 [

### Train CNN + LSTM Hybrid

In [ ]:
checkpoint = ModelCheckpoint("./models/hybrid_model.keras", monitor='val_loss', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

# Define input layer
input_layer = Input(shape=(X_train.shape[1], 1))

# CNN model
conv_layer = Conv1D(filters=32, kernel_size=3, activation='relu')(input_layer)
maxpool_layer = MaxPooling1D(pool_size=2)(conv_layer)
flatten_layer = Flatten()(maxpool_layer)
dense_cnn = Dense(32, activation='relu')(flatten_layer)

# BiLSTM model
lstm_layer = LSTM(64, activation='relu')(input_layer)
# lstm_layer2 = LSTM(32, activation='relu', return_sequences=False)(lstm_layer)
dense_lstm = Dense(32, activation='relu')(lstm_layer)

# Concatenate CNN and BiLSTM outputs
concatenated = Concatenate()([dense_cnn, dense_lstm])

# Output layer
output_layer = Dense(5)(concatenated)

# Create the ensemble model
hybrid_model = Model(inputs=input_layer, outputs=output_layer)

hybrid_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

history = hybrid_model.fit(
    X_train,
    Y_train,
    epochs=50,
    batch_size=512,
    validation_split=0.2,
    callbacks=[checkpoint, early_stopping]
)

loss, mae = hybrid_model.evaluate(X_test, Y_test)
print("Test Mean Absolute Error:", mae)

Y_pred = hybrid_model.predict(X_test)

mae_columns = mean_absolute_error(Y_test, Y_pred, multioutput='raw_values')
print("Mean Absolute Error for each column:")
print(mae_columns)

Epoch 1/50
575/575 [==============================] - ETA: 0s - loss: 409.1539 - mae: 11.3165
Epoch 1: val_loss improved from inf to 387.31973, saving model to ../models\hybrid_model.keras
575/575 [==============================] - 11s 18ms/step - loss: 409.1539 - mae: 11.3165 - val_loss: 387.3197 - val_mae: 9.8553
Epoch 2/50
574/575 [============================>.] - ETA: 0s - loss: 379.9135 - mae: 10.3883
Epoch 2: val_loss improved from 387.31973 to 384.78098, saving model to ../models\hybrid_model.keras
575/575 [==============================] - 10s 17ms/step - loss: 379.9100 - mae: 10.3883 - val_loss: 384.7810 - val_mae: 10.0259
Epoch 3/50
575/575 [==============================] - ETA: 0s - loss: 378.9606 - mae: 10.3539
Epoch 3: val_loss did not improve from 384.78098
575/575 [==============================] - 10s 17ms/step - loss: 378.9606 - mae: 10.3539 - val_loss: 386.5429 - val_mae: 11.1886
Epoch 4/50
573/575 [============================>.] - ETA: 0s - loss: 377.7383 - mae: 1